# 🎨🌈🏞 Crafting Color Images

This page demonstrates how to load RGB color images into a three-dimensional array in Python, how to think about the numbers in that array, and how to construct a color image from three different two-dimensional images. Please work through these examples, discuss with your classmates or instructors, and try the activities on your own.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

### How we load a color image into an array? 

Many of us are probably familiar with lots of different color image formats, including `.jpg`, `.png`, `.gif`, `.tif`, and other. Whereas a single FITS image extension will usually have one brightness value per pixel, these color images store three brightness values per pixel, one each for red, green, blue. 

We can read these image files into arrays providing the `plt.imread` function with the file path to an image. Please copy the file `/home/zkbt/astr3510/rainbow.jpg` into the same directory as your notebook, and then run this code. This function uses the Python Imaging Library (`PIL.Image.open`), so if you need more control over loading images you should work with that directly.

In [ ]:
rainbow = plt.imread('rainbow.jpg')

The variable `rgb` is now a three-dimensional array, with the first dimension corresponding to pixel row, the second to pixel column, and the third to wavelength band. This image has 1434 rows, 1764 columns, and 3 color bands (red, green, blue). 

In [ ]:
rainbow.shape

If our trusty friend `plt.imshow` receives a three-dimensional array with 3 elements in the third dimension, it will try to interpret the array as a color image and display it as such.

In [ ]:
plt.figure(dpi=300)
plt.imshow(rainbow);

This magic trick of treating a 3D array as a color image will work as long as the values in the image are integer numbers between 0 and 255 ($= 2^8 - 1$ for 8-bit color), or floating point numbers between 0.0 and 1.0. Let's look at the RGB values for the first row and first column of our image:

In [ ]:
rainbow[0,0,:]

The data type of the numbers is "unsigned 8-bit integer", meaning whole numbers between 0 and 255. The RGB values for the first row and first pixel are R=184, G=183, B=118, where 0 means no light of that color and 255 means the brightest possible light for that color. If the three values were all the same, the color would be some shade of gray; since there's a little less blue, the color should be shifted a little toward orange-ish. To my eye, the upper left region looks like a warm gray!

## `Discuss!`
Talk to someone:
- Pick at least one pixel in the color image, guess what its RGB pixel brighness values will be, then print out the values to compare. If you can't see some colors, ask your partner(s) to point out what's what.

### What do the individual color channel arrays look like? 
Let's dig into the details of this image a little more. Since we might want to similar actions for a few different images, let's write a function that we can resuse. (The code of this function is available in `/home/zkbt/astr3510/rgb.py`, if you just want to copy and paste it from there.)

In [ ]:
def show_rgb_separately(some_rgb_image, cmap='gray'):
    '''
    This function provides a handy way to look at the 
    individual RGB channels of a color image.
    
    Parameters
    ----------
    some_rgb_image : the color image array to display
        An array with dimensions (rows, columns, 3), with the
        last dimension corresponding to the three RGB colors.
    
    cmap : str
        Which matplotlib colormap should we use for displaying brightness?
        (default = 'gray', with black at bottom and white at top)
    '''
    
    # give names to the colors to use as titles
    colors = ['red', 'green', 'blue']
    
    # check if the units are integer (0-255) or not (0.0-1.0)
    if some_rgb_image.dtype == np.uint8:
        vmax = 255
    else:
        vmax = 1.0
    
    # set up big figure to fill with plots
    fi = plt.figure(figsize=(12,12), dpi=300)

    # set up a grid of panels into which we can plot
    grid = plt.GridSpec(2, 3, height_ratios=[1, 3])
    
    # loop through the three color channels
    for i in range(3):
        
        # point toward the panel in row 0, column i
        plt.subplot(grid[0,i])

        # show the image for this color channel in this panel
        plt.imshow(some_rgb_image[:,:,i], cmap=cmap, vmin=0, vmax=vmax)

        # add a title to the panel
        plt.title(colors[i])
    
    # point toward row 1, all columns
    plt.subplot(grid[1,:])
    
    # show the color image
    plt.imshow(some_rgb_image)

Now let's try out our function on our color image. 

In [ ]:
show_rgb_separately(rainbow)

## `Discuss!`
Talk to someone:
- What is the relationship between the monochrome RGB images and the color image? Pick a region and explain why it has the color it does. 

### How do we construct color images from individual 2D arrays? 

With telescopes, we're often gathering one 2D brightness image at a time. With filters we can select specific wavelengths to make it to the detector, but the data are being recorded as a monochromatic brightness per pixel. To simulate these kinds of images, let's start by making a few imaginary 2D arrays.

In [ ]:
# set up the basic shape of the image 
rows = 10
columns = 12
shape = (rows, columns)

# define some x and y coordinates along the image
x = np.linspace(0, 255, columns)
y = np.linspace(0, 255, rows)

# define 2D arrays that increase along columns and rows
a, b = np.meshgrid(x, y)

# define another 2D full of random numbers
c = np.random.uniform(50, 200, shape)

Next, let's stitch these three 2D arrays into one 3D array that can be interpreted as a color image. There are multiple way to do this, but here we create an empty array with the correct 3D shape, and then populate it color by color.

In [ ]:
# define the appropriate 3D array shape
rgb_shape = (rows, columns, 3)

# create a new empty array with that shape
rgb = np.zeros(rgb_shape)

# populate the color channels one by one
rgb[:,:,0] = a
rgb[:,:,1] = b
rgb[:,:,2] = c

Finally, let's display our new image as a color image. We'll use our `show_rgb_separately` function above so we can see both the individual color frames and the synthesized color image.

In [ ]:
show_rgb_separately(rgb)

Blerg! That color image isn't particularly interesting, and python raises a warning that the input data are being clipped to a range of either 0.0-1.0 for floats (= decimal numbers) or 0-255 for integers (= whole numbers). Indeed, if we print the pixel values below, we see they have decimal numbers that are much higher than 1.0; that means only the very few pixels that have values between 0.0-1.0 are showing up as interesting colors, and the rest are being clipped to the maximum brightness in all channels (= white). 

In [ ]:
rgb

One solution to this problem is to normalize the values so that they all fall between 0.0 and 1.0. Here we're doing a linear normalization set by the maximum value in the image, but any transformation that produces values between 0.0 and 1.0 would work.

In [ ]:
normalized_rgb = rgb/np.max(rgb)
show_rgb_separately(normalized_rgb)

Another solution would be to convert the numbers into integers between 0 and 255. This highlights that `.imread` + `.imshow` allow only 256 levels of brightness to be associated with each color channel. 

In [ ]:
rgb_as_integers = rgb.astype(np.uint8)
show_rgb_separately(rgb_as_integers)

## `Discuss!`
Talk to someone:
- How would we produce an image where all the color brightnesses are fainter by a factor of 2? Try it out!
- How do 3D arrays work? How is pointing to values in them similar to or different from 2D arrays? 

### How do we save color images to files? 

If we want to save the outputs of our `imshow` figures, including labels, titles, multiple panels, or other plot elements, we can use the `savefig` command to save the entire contents of the figure to a file.

In [ ]:
show_rgb_separately(normalized_rgb)
plt.savefig('my-snazzy-image.pdf')

If all we want is the image, we can use the `.imsave` function, which will do similar things as `.imshow` but save directly to a file.

In [ ]:
plt.imsave('my-snazzy-image.jpg', normalized_rgb)

In [ ]:
plt.imsave('my-snazzy-image.png', normalized_rgb)

## How can you apply this yourself? 

With the skills we've explored above, please discuss with your friends and try to do the following:

1. Try to load a different color image into an array. 
2. Make and show a grayscale image where each pixel is the average of its three RGB values.
3. Mess with the colors of an image by swapping which array corresponds to which color.
4. Draw a colorful horizontal line across an image by changing some pixel values in that row.

Colors are confusing *and* arrays are confusing, so please please for help when you need it!